<a href="https://colab.research.google.com/github/cezarviana/fake-news-no-more/blob/main/fake_news_no_more_(verificador_de_noticias).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca google (google_search) para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Priorize fontes jornalísticas reconhecidas e com boa reputação.
        Selecione no máximo 5 lançamentos que demonstrem relevância (baseada na cobertura e qualidade da fonte) e sejam os mais atuais possíveis.
        Se o tópico não possuir 5 notícias a seu respeito, apresente somente as encontradas. Sem adicionar outras notícias com termos parecidos e que não tenham relação direta com o tópico.
        Se o tópico gerar pouca cobertura noticiosa ou reações limitadas, sinalize essa baixa relevância como um possível indicativo para considerar outros tópicos.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca de notícias no Google Search sobre o tópico",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [9]:
################################################
# --- Agente 2: Verificador de Fontes --- #
################################################
def agente_verificador_fontes(topico, lancamentos_buscados):
    verificador_fontes = Agent(
        name="agente_verificador_fontes",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Fontes #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de fontes, especialista em fact-checking.
        Para cada fonte principal identificada:
        - Determine se o site ou canal é conhecido e geralmente considerado confiável.
        - Localize e examine a seção 'Sobre nós' (ou equivalente) para entender a missão, equipe e possíveis vieses do site.
        - Verifique se outras fontes confiáveis corroboram as informações apresentadas pela fonte principal. Liste as fontes que confirmam os achados.
        """,
        description="Agente que verifica as fontes levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fontes = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_fontes = call_agent(verificador_fontes, entrada_do_agente_verificador_fontes)
    return verificacao_fontes

In [10]:
################################################
# --- Agente 3: Verificador de Conteúdo --- #
################################################
def agente_verificador_conteudo(topico, lancamentos_buscados):
    verificador_conteudo = Agent(
        name="agente_verificador_conteudo",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Conteúdo #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de conteúdo, especialista em fact-checking.
        Examine o conteúdo das notícias e informações relacionadas ao tópico.
        Linguagem e Estilo:
        - Avalie se há uso de sofismos ou outras técnicas de persuasão manipuladoras.
        - Identifique erros de ortografia e gramática que possam indicar falta de profissionalismo ou revisão.
        - Distinga claramente entre fatos apresentados e opiniões, verificando se as opiniões são devidamente atribuídas.
        Contexto e Evidências:
        - Verifique se a data da informação é relevante para o contexto atual.
        - Se houver imagens, utilize a busca reversa (Google Imagens) para verificar sua autenticidade e se foram usadas em outros contextos enganosos.
        Coerência:
        - Avalie a lógica interna do conteúdo e sua coerência com informações de outras fontes.
        """,
        description="Agente que verifica o conteúdo das notícias levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_conteudo = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_conteudo = call_agent(verificador_conteudo, entrada_do_agente_verificador_conteudo)
    return verificacao_conteudo

In [11]:
##########################################
# --- Agente 4: Agência de Fact-Checking --- #
##########################################
def agente_verificador_fatos(topico, data_de_hoje):
    verificador_fatos = Agent(
        name="agente_verificador_fatos",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua tarefa é verificar se o tópico/afirmação já foi alvo de checagem por agências de fact-checking confiáveis, através da  busca google (google_search), priorizando as verificações mais recentes sobre o tópico.
        As agências de fact-checking a serem consultadas de acordo com o tópico serão as indicadas abaixo:
        - No Brasil:
          - Aos Fatos;
          - Lupa;
          - UOL Confere;
          - Estadão Verifica;
          - Fato ou Fake (G1);
          - Boatos.org;
          - Agência Pública - Truco no Congresso;
          - Comprova;
          - E-farsas;
          - É isso Mesmo? (O Globo);
          - Portal EBC - Checagem;
        - Internacionais com atuação ou relevância no Brasil:
          - AFP Fact Check;
          - Reuters Fact Check;
          - Snopes;
          - PolitiFact;
          - FactCheck.org;
        - Organizações e Redes Internacionais:
          - International Fact-Checking Network (IFCN);
          - European Fact-Checking Standards Network (EFCSN);
          - Duke Reporters' Lab;
        Informe se o tópico foi encontrado em alguma das agências de fact-checking e qual foi a conclusão dessas agências (verdadeiro, falso, enganoso, etc.).
        Se encontrado, cite a fonte da agência de fact-checking e um breve resumo da sua análise.
        """,
        description="Agente que verifica o que agências de fact-checking dizem a respeito do tópico",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fatos = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    verificacao_fatos = call_agent(verificador_fatos, entrada_do_agente_verificador_fatos)
    return verificacao_fatos

In [12]:
################################################
# --- Agente 5: Organizador do Resultado da Verificação --- #
################################################
def agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos):
    organizador = Agent(
        name="agente_organizador_resultado",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente de Resultados #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é organizar os resultados com base nas análises dos outros agentes sobre o tópico, e determinar a veracidade da informação.
        Apresente um veredicto claro: Verdadeiro, Falso, Enganoso, Insustentável, etc.
        Justifique sua conclusão de forma concisa, utilizando as evidências e os resultados fornecidos pelos outros agentes (agente_buscador, agente_verificador_fontes, agente_verificador_conteudo e agente_verificador_fatos).
        Liste as fontes mais relevantes (sites de notícias confiáveis, agências de fact-checking) que sustentam sua conclusão.
        """,
        description="Agente que organiza os resultados da verificação",
        tools=[google_search]
    )

    entrada_do_agente_organizador_resultado = f"Tópico:{topico}\nLançamentos buscados:{lancamentos_buscados}\nVerificação fontes:{verificacao_fontes}\nVerificação conteúdo:{verificacao_conteudo}\nVerificação fatos:{verificacao_fatos}"
    # Executa o agente
    resultado = call_agent(organizador,  entrada_do_agente_organizador_resultado)
    return resultado

In [13]:

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos pesquisar sobre a veracidade a respeito de {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    verificacao_fontes = agente_verificador_fontes(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Verificador de Fontes) ---\n")
    display(to_markdown(verificacao_fontes))
    print("--------------------------------------------------------------")

    verificacao_conteudo = agente_verificador_conteudo(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 3 (Verificador de Conteúdo) ---\n")
    display(to_markdown(verificacao_conteudo))
    print("--------------------------------------------------------------")

    verificacao_fatos = agente_verificador_fatos(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 4 (Agências de Fact-Checking) ---\n")
    display(to_markdown(verificacao_fatos))
    print("--------------------------------------------------------------")

    resultado = agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos)
    print("\n--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---\n")
    display(to_markdown(resultado))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: terra plana
Maravilha! Vamos pesquisar sobre a veracidade a respeito de terra plana

--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---



> Ok, vou procurar notícias recentes e relevantes sobre o tema "terra plana" usando a ferramenta de busca Google. O foco será em fontes jornalísticas reconhecidas e lançamentos de no máximo um mês antes da data de hoje (17/05/2025).
> 
> 
> Encontrei algumas notícias e artigos que abordam o tema "terra plana", mas a maioria deles não são propriamente notícias de "lançamentos" ou eventos recentes. Em vez disso, são reportagens e artigos que discutem o fenômeno do terraplanismo, refutam as teorias e analisam o crescimento desse tipo de pensamento.
> 
> Considerando o critério de relevância e atualidade (último mês), destaco os seguintes pontos que se encaixam melhor no que foi solicitado:
> 
> 1.  **Declarações de Wiz Khalifa sobre a Terra Plana (Maio de 2025)**: O rapper Wiz Khalifa expressou sua crença na teoria da Terra plana em um podcast, reacendendo o debate sobre o tema e gerando discussões sobre a influência de celebridades e a percepção pública da ciência.
> 2.  **"A Terra é Redonda" (Maio de 2025)**: Artigos de opinião e análises sobre temas variados, incluindo política e sociedade. Embora o título seja relacionado ao tema, o conteúdo não é diretamente sobre o terraplanismo em si.
> 3.  **Turismo na era do reconhecimento facial (Maio 2025)**: Em um mundo cada vez mais dinâmico e conectado, a tecnologia é peça-chave para tornar a experiência turística mais fluida, segura e personalizada.
> 4.  **Publicação excessiva de artigos científicos irrelevantes distorce o debate público (Março 2025)**: O excesso de publicações científicas de baixa qualidade pode levar à perda de confiança na ciência e à formulação de conceitos distorcidos, como a crença na Terra plana.
> 5.  **Terraplanista na Antártida (Dezembro 2024)**: Um influenciador terraplanista viajou para a Antártida para testar suas teorias, mas aparentemente admitiu que estava errado após observar o fenômeno do "Sol da meia-noite".
> 
> **Considerações Adicionais:**
> 
> *   A maioria dos resultados encontrados são artigos que refutam a teoria da Terra plana, o que indica que o tema ainda gera interesse e debate, mesmo que a comunidade científica já tenha estabelecido a forma esférica da Terra.
> *   Há artigos que analisam o fenômeno do terraplanismo sob a perspectiva da pós-verdade e do negacionismo científico, destacando o papel das redes sociais na disseminação dessas ideias.
> *   Alguns artigos mencionam figuras públicas e celebridades que defendem a teoria da Terra plana, o que contribui para a sua divulgação e alcance.
> *   Um artigo relata a experiência de um ex-terraplanista que viajou para a Noruega para comprovar suas teorias, mas acabou se convencendo de que a Terra é redonda.
> 
> Em resumo, embora não haja muitos "lançamentos" recentes relacionados à Terra plana, o tema continua a ser discutido e analisado em diferentes contextos, principalmente no que diz respeito à desinformação e ao negacionismo científico.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Verificador de Fontes) ---



> Ok, com base nos resultados da busca, vou verificar as fontes principais identificadas para determinar a confiabilidade e possíveis vieses, e procurar outras fontes que corroborem as informações apresentadas. As fontes principais que identifiquei são:
> 
> 1.  **Notícias sobre as declarações de Wiz Khalifa sobre a Terra Plana:** Preciso identificar qual veículo noticiou essa declaração e verificar sua confiabilidade.
> 2.  **Artigo "A Terra é Redonda":** Verificar a fonte deste artigo e determinar sua credibilidade.
> 3.  **Artigo sobre Turismo na era do reconhecimento facial:** Verificar a fonte deste artigo para determinar sua credibilidade.
> 4.  **Artigo sobre a publicação excessiva de artigos científicos irrelevantes:** Identificar a fonte e avaliar sua confiabilidade.
> 5.  **Relato da experiência do terraplanista na Antártida:** Identificar a fonte original deste relato e verificar se há outras fontes corroborando a história.
> 
> Para cada uma dessas fontes, vou procurar a seção "Sobre nós" (ou equivalente) para entender a missão, equipe e possíveis vieses. Também vou procurar outras fontes confiáveis que confirmem as informações apresentadas.
> 
> 
> Com base nos resultados da pesquisa, aqui está a verificação das fontes identificadas:
> 
> 1.  **Declarações de Wiz Khalifa sobre a Terra Plana:**
> 
> *   **Fonte:** Vários sites de notícias e entretenimento noticiaram as declarações de Wiz Khalifa no "The Joe Budden Podcast". Um deles é o Vinyl Me, Please (vmp).
> *   **Confiabilidade:** Vinyl Me, Please (vmp) é uma comunidade de discos de vinil e loja online que também publica artigos e notícias sobre música. A informação parece ser precisa, já que vários outros veículos também reportaram a mesma declaração.
> *   **Corroboração:** As declarações de Wiz Khalifa foram amplamente divulgadas por outros meios de comunicação, como o Portal Piaui do Povo e outros. A contradição entre a crença de Wiz Khalifa e a capa de seu álbum com uma imagem da Terra redonda também foi notada.
> 
> 2.  **Artigo "A Terra é Redonda":**
> 
> *   **Fonte:** A Terra é Redonda.
> *   **Confiabilidade:** "A Terra é Redonda" é um site de notícias e análises com foco em política, filosofia, história e cultura. O site publica artigos de opinião e análises de diversos autores.
> *   **Corroboração:** O título "A Terra é Redonda" é uma afirmação factual bem estabelecida e corroborada por inúmeras fontes científicas. O próprio site parece ter um viés em relação à defesa da ciência e do conhecimento estabelecido.
> 
> 3.  **Artigo sobre Turismo na era do reconhecimento facial:**
> 
> *   **Fonte:** UNINTER NOTÍCIAS.
> *   **Confiabilidade:** UNINTER NOTÍCIAS é um portal de notícias da Universidade Internacional (Uninter). O site divulga informações sobre a instituição, seus cursos, professores e alunos.
> *   **Corroboração:** A notícia sobre o uso de reconhecimento facial em aeroportos brasileiros pode ser confirmada por outras fontes de notícias sobre tecnologia e turismo. A Uninter Notícias parece ser uma fonte confiável para informações sobre as atividades da universidade e temas relacionados à educação e tecnologia.
> 
> 4.  **Artigo sobre a publicação excessiva de artigos científicos irrelevantes:**
> 
> *   **Fonte:** RESICLIMA.
> *   **Confiabilidade:** RESICLIMA é uma plataforma que divulga artigos e notícias sobre sustentabilidade, meio ambiente e ciência. O artigo em questão foi republicado da Revista Questão de Ciência.
> *   **Corroboração:** O tema da publicação excessiva de artigos científicos e seu impacto na credibilidade da ciência é um debate relevante na comunidade acadêmica. A Revista Questão de Ciência é uma fonte confiável para artigos de opinião e análise sobre temas científicos.
> 
> 5.  **Relato da experiência do terraplanista na Antártida:**
> 
> *   **Fonte:** UOL, Superinteressante, e outros.
> *   **Confiabilidade:** UOL e Superinteressante são veículos de comunicação bem estabelecidos e geralmente confiáveis.
> *   **Corroboração:** A história do terraplanista que viajou para a Antártida para testar suas teorias e admitiu que estava errado foi amplamente divulgada. O experimento "O Experimento Final" e a participação de Jeran Campanella foram documentados em vídeo.
> 
> Em resumo, as fontes identificadas parecem ser geralmente confiáveis, e as informações apresentadas são corroboradas por outras fontes. No entanto, é importante notar que alguns dos artigos são de opinião e podem refletir os vieses de seus autores ou veículos de comunicação. É sempre recomendável consultar diversas fontes e verificar as informações antes de tirar conclusões.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Verificador de Conteúdo) ---



...

--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Agências de Fact-Checking) ---



> Sim, o tópico "Terra Plana" foi alvo de checagens por diversas agências de fact-checking confiáveis. A conclusão unânime dessas agências é que a teoria da Terra Plana é falsa e se baseia em desinformação e teorias da conspiração.
> 
> **Exemplos de Checagens:**
> 
> *   **Boatos.org:** Desmentiu diversos boatos relacionados à Terra Plana, incluindo alegações de que a Bíblia e experimentos comprovam que a Terra é plana. A agência classifica essas alegações como "fake news conspiracionistas".
> *   **UOL Confere:** Publicou artigos refutando a teoria da Terra Plana, apresentando evidências científicas que comprovam o formato esférico da Terra.
> *   **AFP Fact Check e Reuters Fact Check:** Também desmentiram alegações sobre a Terra plana.
> *   **Agência Lupa:** Publicou conteúdos que mostram que a Terra não é plana, e sim redonda.
> *   **Estadão Verifica:** O Estadão Verifica já fez um compilado de dicas sobre como identificar conteúdos gerados por inteligência artificial e explicações de como ela funciona.
> *   **IFSC Verifica:** Afirma que o consenso científico de que a Terra é redonda já tem mais de 2 mil anos.
> 
> **Análise:**
> 
> As agências de fact-checking utilizam diversas metodologias para desmentir a teoria da Terra Plana, incluindo:
> 
> *   Apresentação de evidências científicas, como fotos de satélite, vídeos de astronautas e experimentos físicos.
> *   Refutação de argumentos terraplanistas com base em princípios científicos e observações empíricas.
> *   Contextualização da teoria da Terra Plana como uma conspiração pseudocientífica.
> 
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---



> Com base nas análises dos agentes, o tópico "Terra Plana" é classificado como **Falso**.
> 
> **Justificativa:**
> 
> *   A teoria da Terra Plana é amplamente refutada pela comunidade científica, que apresenta evidências irrefutáveis do formato esférico da Terra.
> *   Agências de fact-checking como Boatos.org, UOL Confere, AFP Fact Check, Reuters Fact Check, Agência Lupa, Estadão Verifica e IFSC Verifica desmentiram alegações sobre a Terra Plana, classificando-as como desinformação e teorias da conspiração.
> *   Embora o tema continue a gerar discussões e análises, principalmente no contexto da desinformação e do negacionismo científico, a veracidade da teoria da Terra Plana é nula.
> 
> **Fontes Relevantes:**
> 
> *   Boatos.org
> *   UOL Confere
> *   AFP Fact Check
> *   Reuters Fact Check
> *   Agência Lupa
> *   Estadão Verifica
> *   IFSC Verifica
> 


--------------------------------------------------------------
